In [3]:
import pickle
import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def FCuCorrelation(FC1, FC2, fisher=True):
    u_idx = np.triu_indices_from(FC1, k=1)
    FCu1 = FC1.copy()[u_idx]
    FCu2 = FC2.copy()[u_idx]
    if fisher:
        FCu1 = np.arctanh(FCu1)
        FCu2 = np.arctanh(FCu2)
    FC_corr_upper = np.corrcoef(FCu1, FCu2)[0, 1]
    return FC_corr_upper

demo_data = pd.read_csv(f"/Users/brainsimulation/Desktop/NMDARE_FC_Analysis/out/DesikanKilliany/demo_data.csv")

derivatives_dir = "/Users/brainsimulation/Desktop/derivatives_dk_T2uniform/derivatives"

excluded = ['RSGK017', 'LENMDA001', 'LENMDA006', 'LENMDA172', 'LENMDA187', 'LENMDA191']
subids = [subid[4:] for subid in os.listdir(os.path.join(derivatives_dir, "xcp_d")) if os.path.exists(os.path.join(derivatives_dir, "xcp_d", f"{subid}", "func", f"{subid}_task-rest_space-fsLR_seg-DesikanKilliany_den-91k_stat-pearsoncorrelation_boldmap.pconn.nii")) and subid[4:] not in excluded]


In [ ]:
window_size = 30
step_size = 1

nmdare_FCDs = []
nmdare_hists = []
hc_FCDs = []
hc_hists = []

with open(f"/Users/brainsimulation/Desktop/NMDARE_FC_Analysis/out/DesikanKilliany/to_delete_network.pkl", "rb") as file:
    to_delete = pickle.load(file)

for subid in subids:
    xcp_func_folder = os.path.join(derivatives_dir, "xcp_d", f"sub-{subid}", "func")

    ts_fname = os.path.join(xcp_func_folder, f"sub-{subid}_task-rest_space-fsLR_seg-DesikanKilliany_den-91k_stat-mean_timeseries.ptseries.nii")
    ts = np.array(nib.load(ts_fname).dataobj)

    n_windows = len(range(0, ts.shape[0] - window_size, step_size))

    sub_fcs = []
    for i in range(0, ts.shape[0] - window_size, step_size):
        window_ts = ts[i:i + window_size, :]
        window_ts = np.delete(window_ts, to_delete, 1)
        window_ts = window_ts[:66]

        fc = np.corrcoef(window_ts, rowvar=False)
        np.fill_diagonal(fc, np.nan)
        sub_fcs.append(fc)
    
    n_fcs = len(sub_fcs)

    FCD = np.zeros((n_fcs, n_fcs))
    for i in range(n_fcs):
        for j in range(n_fcs):
            FCD[i, j] = FCuCorrelation(sub_fcs[i], sub_fcs[j], fisher=True)
    
    hist = np.histogram(FCD, 100)[0]

    os.makedirs(f"{subid}", exist_ok=True)
    np.savetxt(f"{subid}/FCD.csv", FCD, delimiter=',')
    np.savetxt(f"{subid}/hist.csv", hist, delimiter=',')

    if demo_data.loc[demo_data["ID"] == subid]["nmdare"].item() == 1:
        nmdare_FCDs.append(FCD)
        nmdare_hists.append(hist)
    else:
        hc_FCDs.append(FCD)
        hc_hists.append(hist)
    
np.savetxt("nmdare_FCDs.csv", nmdare_FCDs, delimiter=',')
np.savetxt("hc_FCDs.csv", hc_FCDs, delimiter=',')
np.savetxt("nmdare_hists.csv", nmdare_hists, delimiter=',')
np.savetxt("hc_hists.csv", hc_hists, delimiter=',')

/var/folders/mn/q624n0lj15s26j52tyhh602h0000gn/T/ipykernel_3196/2236525035.py:13: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  ts = np.array(nib.load(ts_fname).dataobj)


ValueError: autodetected range of [nan, nan] is not finite

In [9]:
sub_fcs[10].shape

(112, 112)